In [1]:
import datetime

In [2]:
%%cython
def usp_comp(amat):
    """ Return the size of the smallest unique-shortest-path complement of a graph.
    """
    # Accept graph, graph6 string, or matrix input
    
    try:
        nn = amat.ncols()
    except AttributeError:
        amat = amat.adjacency_matrix()
        nn = amat.ncols()
    if not amat:
        return nn - 1
    A = amat + 2
    AA = A + 0
    compsize = nn - 1
    while min(min([yy for yy in xx if yy]) for xx in AA) == 1:
        compsize -= 1
        AA = AA*A
    
    return compsize

In [3]:
import gc

gc.enable()

In [4]:
def Glabel(G):
    return G.canonical_label().graph6_string()


def edgeclasses(G):
    Aut = G.automorphism_group()
    needs = {(xx[0], xx[1]): True for xx in G.edges()}
    while needs:
        anedge = next(iter(needs))
        yield anedge
        for xx in Aut.orbit(anedge, action='OnPairs'):
            if (xx[0], xx[1]) in needs:
                del needs[(xx[0], xx[1])]
            if (xx[1], xx[0]) in needs:
                del needs[(xx[1], xx[0])]

                
def deletions(G):
    for ed in edgeclasses(G):
        H = G.copy()
        H.delete_edge(ed)
        yield Glabel(H)

        
def contractions(G):
    # Also covers isolated vertex deletion
    for ed in edgeclasses(G):
        H = G.copy()
        H.contract_edge(ed)
        yield Glabel(H)
    if 0 in G.degree():
        H = G.copy()
        H.delete_vertex(G.degree().index(0))
        yield Glabel(H)

In [5]:
def read_prior_minors(previous_nn, previous_edge_count):
    import ast
    
    path_prefix = 'minor_minimal_graphs'
    
    uspcm_list = []

    with open(path_prefix + f'/uspcm_dict/full_uspcm_dict_after_n_{previous_nn}_edges-{previous_edge_count}.txt', 'r') as infile:
        stored_uspcm = infile.read()

        uspcm_list.extend(ast.literal_eval(stored_uspcm))


    minimals_list = []

    with open(path_prefix + f'/all_minimals/all_minimals_after_n_{previous_nn}_edges-{previous_edge_count}.txt', 'r') as infile:
        stored_minimals = infile.read()

        minimals_list.extend(ast.literal_eval(stored_minimals))
        
    return uspcm_list, minimals_list


def write_computed_minors(nn, edge_count, uspcm_dict, minimals):
    # Craig added this to save to files
    path_prefix = 'minor_minimal_graphs'
    
    with open(path_prefix + 
              f'/all_minimals/all_minimals_after_n_{nn}_edges-{edge_count}.txt', 'w') as outfile:
        outfile.write(str(minimals))

    with open(path_prefix + 
              f'/uspcm_dict/full_uspcm_dict_after_n_{nn}_edges-{edge_count}.txt', 'w') as outfile:
        outfile.write(str(uspcm_dict))
        
        
def write_computed_data_v2(nn, edge_count, uspcm_dict, minimals):
    # Craig added this to save to files
    path_prefix = 'minor_minimal_graphs'
    with open(path_prefix +
              f'/all_minimals/all_minimals_after_n_{nn}_edges-{edge_count}.txt', 'w') as outfile:
        outfile.write(str(minimals))

    with open(path_prefix +
              f'/uspcm_dict/full_uspcm_dict_after_n_{nn}_edges-{edge_count}.txt', 'w') as outfile:
        outfile.write(str(uspcm_dict))

In [6]:
# def exhaustive_search_range_nn(previous_nn, last_nn_to_do):
#     previous_edge_count = 1
#     path_prefix = 'minor_minimal_graphs'

#     if previous_nn > 1:
#         # Import previously computed uspc_dict and minimals
#         uspcm_dict, minimals = read_prior_minors(previous_nn, previous_edge_count)
#     elif previous_nn == 1:
#         uspcm_dict = [[{Glabel(Graph(0)): 0}], [{Glabel(Graph(1)): 0}]]
#         minimals = [[Graph(Glabel(Graph(1))).graph6_string()]]

#     for nn in range(previous_nn+1, last_nn_to_do+1):

#         # Craig added to write to file
#         with open(path_prefix + f'/minimal_for_n_{nn}.txt', 'w') as outfile:


#             print(f'Working on graphs with {nn} vertices.')
#             G = Graph(nn).complement() # Graph(nn) is empty graph on nn vertices so Graph(nn).complement() is K_n
#             uspcm_dict.append([{Glabel(G): nn - 2}])
#             edcount = Integer((nn*(nn-1))/2)
#             while edcount:
#                 print(f'{edcount} edges:')
#                 uspcm_dict[-1].append(dict())
#                 # First pass: set values, giving every superminor a chance.
#                 for amat in uspcm_dict[-1][-2]:
#                     mine = uspcm_dict[-1][-2][amat]
#                     G = Graph(amat)
#                     for xx in deletions(G):
#                         if xx in uspcm_dict[-1][-1]:
#                             # print(f'Updating {Graph(xx).graph6_string()} from {uspcm_dict[-1][-1][xx]}', end='')
#                             # print(f' to {mine}.')
#                             old = uspcm_dict[-1][-1][xx]
#                             uspcm_dict[-1][-1][xx] = min(mine, old)
#                         else:
#                             # print(f'Setting {Graph(xx).graph6_string()} to {usp_comp(xx)}.')
#                             old = usp_comp(Graph(xx))
#                             uspcm_dict[-1][-1][xx] = min(mine, old)
#                     for xx in contractions(G):
#                         edplace = Integer((nn - 1)*(nn - 2)/2) - Graph(xx).size()
#                         old = uspcm_dict[-2][edplace][xx]
#                         if old > mine:
#                             print(f'Exception found: {G.graph6_string()} has uspc {mine}', end='')
#                             print(f' with minor {Graph(xx).graph6_string()} of uspc {uspcm_dict[-2][edplace][xx]}.')
#                 # Second pass: check for minimality
#                 for amat in uspcm_dict[-1][-2]:
#                     mine = uspcm_dict[-1][-2][amat]
#                     G = Graph(amat)
#                     for xx in deletions(G):
#                         if mine == uspcm_dict[-1][-1][xx]:
#                             break
#                     else:
#                         for xx in contractions(G):
#                             edplace = Integer((nn - 1)*(nn - 2)/2) - Graph(xx).size()
#                             if mine == uspcm_dict[-2][edplace][xx]:
#                                 break
#                         else:
#                             print(f'Minor minimal for {mine}: {G.graph6_string()}.')

#                             # Craig added this to write to also write to file
#                             outfile.write(f'Minor minimal for {mine}: {G.graph6_string()}.\n')

#                             while len(minimals) < mine + 1:
#                                 minimals.append([])
#                             minimals[mine].append(G.graph6_string())
                            
                            
#                 # Write/save the computed results for |V(G)| = nn, |E(G)| = edcount
#                 write_computed_minors(nn, edcount, uspcm_dict, minimals)
                
#                 edcount -= 1
#         print('Done.')

In [7]:
# exhaustive_search_range_nn(previous_nn=1, last_nn_to_do=6)

In [8]:
# def find_latest_worked_size():
#     import os, re
#     path_prefix = 'minor_minimal_graphs'
    
#     worked_Ns = []
#     worked_Edges = []

#     for filename in os.listdir(path_prefix + '/all_minimals'):


#         if 'minimals_after_n_' in filename:
#             worked_Ns.append(int(re.findall('_n_[0-9]', filename)[0].split('_')[-1]))
#             worked_Edges.append(int(re.findall('edges-[0-9]+', filename)[0].split('-')[-1]))


#     worked_vertices_edges = list(zip(worked_Ns, worked_Edges))

#     max_worked_N = min(max(worked_Ns), nn)
#     worked_edges_for_max_N = []
#     for N, E in worked_vertices_edges:
#         if N == max(worked_Ns):
#             worked_edges_for_max_N.append(E)

#     min_E_for_max_N = min(worked_edges_for_max_N)
    
#     return (max_worked_N, min_E_for_max_N)



# def get_previously_computed(max_worked_N, min_worked_E):
#     import ast
    
#     path_prefix = 'minor_minimal_graphs'
    
#     uspcm_list = []

#     with open(path_prefix +
#               f'/uspcm_dict/full_uspcm_dict_after_n_{max_worked_N}_edges-{min_worked_E}.txt', 'r') as infile:
#         stored_uspcm = infile.read()

#         uspcm_list.extend(ast.literal_eval(stored_uspcm))


#     minimals_list = []

#     with open(path_prefix +
#               f'/all_minimals/all_minimals_after_n_{max_worked_N}_edges-{min_worked_E}.txt', 'r') as infile:
#         stored_minimals = infile.read()

#         minimals_list.extend(ast.literal_eval(stored_minimals))
        
#     return uspcm_list, minimals_list




def get_status():
    import ast
    
    path_prefix = 'minor_minimal_graphs'
    try:
        with open(path_prefix + '/completed_status.txt', 'r') as infile:
            status_dict = ast.literal_eval(infile.read())
    except FileNotFoundError:
        status_dict = {}
        
    return status_dict

def update_status(status_dict):
    path_prefix = 'minor_minimal_graphs'

    with open(path_prefix + '/completed_status.txt', 'w') as outfile:
        outfile.write(status_dict)
        
        
def get_last_status(nn, status_dict):
    vert_edge_tuples = list(status_dict.keys())
    
    last_tup = sorted(vert_edge_tuples, key=lambda x: (-x[0], x[1]))[0]

    if len(status_dict[last_tup]) == 1:
        return (last_tup, ' '.join(status_dict[last_tup][0].split()[0:2]))
    elif len(status_dict[last_tup]) == 2:
        return (last_tup, ' '.join(status_dict[last_tup][1].split()[0:2]))
    
    
    
def get_previously_computed(last_worked_N, last_worked_E):
    import ast
    
    path_prefix = 'minor_minimal_graphs'
    
    uspcm_list = []

    with open(path_prefix +
              f'/uspcm_dict/full_uspcm_dict_after_n_{last_worked_N}_edges-{last_worked_E}.txt', 'r') as infile:
        stored_uspcm = infile.read()

        uspcm_list.extend(ast.literal_eval(stored_uspcm))


    minimals_list = []

    with open(path_prefix +
              f'/all_minimals/all_minimals_after_n_{last_worked_N}_edges-{last_worked_E}.txt', 'r') as infile:
        stored_minimals = infile.read()

        minimals_list.extend(ast.literal_eval(stored_minimals))
        
    return uspcm_list, minimals_list



def progressBar(iterable, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    import datetime
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    total = len(iterable)
    # Progress Bar Printing Function
    def printProgressBar (iteration):
        percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} {prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Initial Call
    printProgressBar(0)
    # Update Progress Bar
    for i, item in enumerate(iterable):
        yield item
        printProgressBar(i + 1)
    # Print New Line on Complete
    print()
    
    
    
    
def first_pass(nn): # for some reason nn was taking on a different value if I used it as a global variable without passing it as a parameter
    print(f'Edcount after entering first_pass() function: {edcount}')
    
    one_percent = len(uspcm_dict[-1][-2]) // 100
    num_graphs_worked = 0
    
    # First pass: set values, giving every superminor a chance.
    for amat in progressBar(uspcm_dict[-1][-2], 
                            prefix = f'Progress on 1st pass:', 
                            suffix = 'Complete', length = 50):

        mine = uspcm_dict[-1][-2][amat]
        G = Graph(amat)
        for xx in deletions(G):
            if xx in uspcm_dict[-1][-1]:
                # print(f'Updating {Graph(xx).graph6_string()} from {uspcm_dict[-1][-1][xx]}', end='')
                # print(f' to {mine}.')
                old = uspcm_dict[-1][-1][xx]
                uspcm_dict[-1][-1][xx] = min(mine, old)
            else:
                # print(f'Setting {Graph(xx).graph6_string()} to {usp_comp(xx)}.')
                old = usp_comp(Graph(xx))
                uspcm_dict[-1][-1][xx] = min(mine, old)
        for xx in contractions(G):
            edplace = Integer((nn - 1)*(nn - 2)/2) - Graph(xx).size()
            old = uspcm_dict[-2][edplace][xx]
            if old > mine:
                print(f'Exception found: {G.graph6_string()} has uspc {mine}', end='')
                print(f' with minor {Graph(xx).graph6_string()} of uspc {uspcm_dict[-2][edplace][xx]}.')
                
        
        # Save to file every one percent of the way through
        num_graphs_worked += 1
        if num_graphs_worked % one_percent == 0:
            write_computed_minors(nn, edcount, uspcm_dict, minimals)
        
        

def second_pass(nn):
    
    one_percent = len(uspcm_dict[-1][-2]) // 100
    num_graphs_worked = 0
    
    # Second pass: check for minimality
    for amat in progressBar(uspcm_dict[-1][-2], 
                        prefix = f'Progress on 2nd pass, checking for minimality:', 
                        suffix = 'Complete', length = 50):
        mine = uspcm_dict[-1][-2][amat]
        G = Graph(amat)
        for xx in deletions(G):
            if mine == uspcm_dict[-1][-1][xx]:
                break
        else:
            for xx in contractions(G):
                edplace = Integer((nn - 1)*(nn - 2)/2) - Graph(xx).size()
                if mine == uspcm_dict[-2][edplace][xx]:
                    break
            else:
                print(f'Minor minimal for {mine}: {G.graph6_string()}.')

                # Craig added this to write to also write to file
                with open(path_prefix + f'/minimal_for_n_{nn}.txt', 'a') as outfile:
                    outfile.write(f'Minor minimal for {mine}: {G.graph6_string()}.\n')

                while len(minimals) < mine + 1:
                    minimals.append([])
                minimals[mine].append(G.graph6_string())


        # Save to file every one percent of the way through
        num_graphs_worked += 1
        if num_graphs_worked % one_percent == 0:
            write_computed_minors(nn, edcount, uspcm_dict, minimals)


    # Update status
    status_dict[(nn, edcount)].append('Second Pass Done')
    update_status(str(status_dict))

In [9]:
def exhaustive_search_limited_edge_counts(nn, first_edge_count, last_edge_count_to_do, 
                                          load_from_previous_nn=False, force_computation=True):
    import os
    
    global path_prefix
    global status_dict    
    global uspcm_dict
    global minimals
    global edcount
    global max_worked_N
    
    path_prefix = 'minor_minimal_graphs'
    if 'all_minimals' not in os.listdir(path_prefix):
        os.mkdir(path_prefix + '/all_minimals')
    if 'uspcm_dict' not in os.listdir(path_prefix):
        os.mkdir(path_prefix + '/uspcm_dict')
    
    status_dict = get_status()
    last_status = get_last_status(nn, status_dict)
    
    
    if last_status[0][0] >= nn and last_status[0][1] <= first_edge_count and last_status[1] == 'Second Pass':
        if force_computation:
            if first_edge_count == 0:
                last_status = ((nn-1, 1), 'Second Pass')
            else:
                last_status = ((nn, first_edge_count), 'Second Pass')
        else:
            return None
    
    
#     if last_status[0][0] > nn or (last_status[0][0] == nn and last_status[0][1] >= edge_count):
#         last_status = ((nn, edge_count+1), 'Second Pass')
    print(last_status)
    skip_first_pass = False
    try:
        if last_status[1] == 'First Pass':
            skip_first_pass = True
    except:
        pass

    
    if nn <= 2:
        uspcm_dict = [[{Glabel(Graph(0)): 0}], [{Glabel(Graph(1)): 0}]]
        minimals = [[Graph(Glabel(Graph(1))).graph6_string()]]
        edcount = 1
        
    elif load_from_previous_nn == True:
        # Import previously computed uspc_dict and minimals
#         max_worked_N, edcount = find_latest_worked_size()
        last_worked_N, edcount = last_status[0]
        uspcm_dict, minimals = get_previously_computed(last_worked_N, edcount)
    

    print(f'Working on graphs with {nn} vertices.')
    if edcount == 1:
        G = Graph(nn).complement() # Graph(nn) is empty graph on nn vertices so Graph(nn).complement() is K_n
        uspcm_dict.append([{Glabel(G): nn - 2}])
        edcount = Integer((nn*(nn-1))/2)
    else:
        edcount = edcount - 1

    while edcount >= last_edge_count_to_do:
        print(f'\n{edcount} edges:')
        uspcm_dict[-1].append(dict())


        # For informational purposes only
        num_to_check = len(uspcm_dict[-1][-2])
        print(f'Need to go through {num_to_check} graphs on {nn} vertices with {edcount} edges.\nStart time: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.')

# Still need to modify the code to only go through graphs that haven't already been computed
# graphs_left_to_check_list = list(set(graph6_list) - set(worked_graphs_list))
        
        if skip_first_pass:
            continue
        else:
            print(f'Edge count before entering first_pass() function: {edcount}')
            first_pass(nn)
            
#             # Update status
#             status_dict.update({(nn, edcount):['First Pass Done']})
#             update_status(str(status_dict))


#         second_pass(nn)


        edcount -= 1
    print('\n\nAll Done!\n\n')

In [10]:
# exhaustive_search_limited_edge_counts(nn=8, previous_edge_count=6, last_edge_count_to_do=1, load_from_previous_nn=True)

In [11]:
# exhaustive_search_limited_edge_counts(nn=9, previous_edge_count=1, last_edge_count_to_do=19, load_from_previous_nn=True)

In [12]:
# exhaustive_search_limited_edge_counts(nn=9, previous_edge_count=19, last_edge_count_to_do=1)

In [13]:
# exhaustive_search_limited_edge_counts(nn=10, previous_edge_count=1, last_edge_count_to_do=40, load_from_previous_nn=True)

In [14]:
# for nn in range(1,9):
#     exhaustive_search_limited_edge_counts(nn, previous_edge_count=1, last_edge_count_to_do=1,
#                                           load_from_previous_nn=True, force_computation=True)

In [15]:
previous_edge_count = get_last_status(10, get_status())[0][1]
next_edge_count = previous_edge_count - 1
next_edge_count

25

In [17]:
exhaustive_search_limited_edge_counts(10, first_edge_count=next_edge_count,
                                      last_edge_count_to_do=next_edge_count-3,
                                      load_from_previous_nn=True, force_computation=True)

((10, 26), 'Second Pass')
Working on graphs with 10 vertices.

25 edges:
Need to go through 1061159 graphs on 10 vertices with 25 edges.
Start time: 2021-06-13 02:07:04.
Edge count before entering first_pass() function: 25
Edcount after entering first_pass() function: 25


KeyboardInterrupt: 